In [ ]:
# =============================== 
#  Library Installation
# ===============================
!pip install numpy==1.26.4 scipy==1.11.4 scikit-learn==1.3.2 opencv-python==4.8.1.78
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu
!pip install ultralytics==8.3.24 

# ===============================

import cv2
from ultralytics import YOLO
import time
from IPython.display import display, clear_output, Video
from PIL import Image
import numpy as np

# ===============================
#  YOLO Setup
# ===============================
model = YOLO("yolov8m.pt")  # Load medium-size model
Classes = model.model.names
TargetClasses = {k: v for k, v in Classes.items() if v in ["car", "bus", "truck"]}
ClsInx = list(TargetClasses.keys())

# ===============================
#  Load Video
# ===============================
video_path = "traffictrim.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"The video was not found: {video_path}")

# ===============================
#  Variables Setup
# ===============================
line1_y = 250
line2_y = 400
speed_limit = 60
car_dict = {}
TotalCounter = set()

# ===============================
#  Output Video Writer Setup
# ===============================
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# ===============================
#  Tracking and Processing
# ===============================
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print(" Video processing completed")
        break

    frame_count += 1

    # Draw lines
    cv2.line(frame, (0, line1_y), (width, line1_y), (255, 0, 0), 2)
    cv2.line(frame, (0, line2_y), (width, line2_y), (0, 0, 255), 2)

    # YOLO tracking
    results = model.track(frame, persist=True, classes=ClsInx, conf=0.3, show=False)

    for result in results:
        if result.boxes.id is None:
            continue

        boxes = result.boxes.xyxy.cpu().numpy()
        ids = result.boxes.id.cpu().numpy().astype(int)
        classes = result.boxes.cls.cpu().numpy().astype(int)

        for box, id_, cls in zip(boxes, ids, classes):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Vehicle center

            if int(cls) in ClsInx:

                if id_ not in car_dict:
                    car_dict[id_] = {'entered': False, 'passed': False, 'time': None, 'speed': None}

                # Cross first line
                if cy >= line1_y and not car_dict[id_]['entered']:
                    car_dict[id_]['time'] = time.time()
                    car_dict[id_]['entered'] = True

                # Cross second line
                if car_dict[id_]['entered'] and not car_dict[id_]['passed'] and cy >= line2_y:
                    elapsed_time = time.time() - car_dict[id_]['time']
                    speed = 150 / elapsed_time  # Estimated speed in px/sec
                    car_dict[id_]['speed'] = speed
                    car_dict[id_]['passed'] = True
                    TotalCounter.add(id_)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID:{id_}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                # Speed display and warning
                if car_dict[id_]['speed'] is not None:
                    sp = int(car_dict[id_]['speed'])
                    color = (0, 255, 0) if sp <= speed_limit else (0, 0, 255)
                    cv2.putText(frame, f"Speed:{sp}px/s", (x1, y1 - 35),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    if sp > speed_limit:
                        cv2.putText(frame, "OVERSPEED!", (x1, y1 - 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Count vehicles
    cv2.putText(frame, f"Total Vehicles: {len(TotalCounter)}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

    # Save frame
    out.write(frame)

    # Display live update in Jupyter
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(frame_rgb))
    clear_output(wait=True)

# ===============================
#  End Video
# ===============================
cap.release()
out.release()
print("\n Output video saved as: output.avi")

# ===============================
#  Display Output Video 
# ===============================
Video("output.avi", embed=True, width=640, height=480)
